In [1]:
from hey_ac import HeyAC

In [2]:
h = HeyAC(grammar_path = '../grammars/grammar_rom.txt')

In [3]:
texts = [
    'the speed of the fan is too low',
    'fan speed is low',
    'temperature is high',
    'humidity feels too high',
    'The room is very damp',
    'It is burning',
    'it is so cold',
    'the fan is not quite fast',
    'i feel very hot',
    'i am freezing',
    'the dog fe'
]

In [4]:
for text in texts:
    print("="*50)
    print(text)
    print(h.prune(text))

the speed of the fan is too low
{'NN_PROP': 'SPEED', 'NN_OBJ': 'FAN', 'VB': [], 'NEG': [], 'JJ': 'low', 'USER': []}
fan speed is low
{'NN_PROP': 'SPEED', 'NN_OBJ': 'FAN', 'VB': [], 'NEG': [], 'JJ': 'low', 'USER': []}
temperature is high
{'NN_PROP': 'TEMPERATURE', 'NN_OBJ': [], 'VB': [], 'NEG': [], 'JJ': 'high', 'USER': []}
humidity feels too high
{'NN_PROP': 'HUMIDITY', 'NN_OBJ': [], 'VB': [], 'NEG': [], 'JJ': 'high', 'USER': []}
The room is very damp
{'NN_PROP': [], 'NN_OBJ': 'ENVIRONMENT', 'VB': [], 'NEG': [], 'JJ': 'HUMID', 'USER': []}
It is burning
{'NN_PROP': [], 'NN_OBJ': 'ENVIRONMENT', 'VB': [], 'NEG': [], 'JJ': 'HOT', 'USER': []}
it is so cold
{'NN_PROP': [], 'NN_OBJ': 'ENVIRONMENT', 'VB': [], 'NEG': [], 'JJ': 'COLD', 'USER': []}
the fan is not quite fast
{'NN_PROP': [], 'NN_OBJ': 'FAN', 'VB': [], 'NEG': 'not', 'JJ': 'FAST', 'USER': []}
i feel very hot
{'NN_PROP': [], 'NN_OBJ': [], 'VB': [], 'NEG': [], 'JJ': 'HOT', 'USER': 'i'}
i am freezing
{'NN_PROP': [], 'NN_OBJ': [], 'VB': 

In [5]:
for text in texts:
    print("="*50)
    print(text)
    print(h.parse(text)[0])

the speed of the fan is too low
(S
  (NP
    (PROPERTY (DT the) (NN_PROP (SPEED speed)))
    (IN of)
    (OBJECT (DT the) (NN_OBJ (FAN fan))))
  (VP (VBZ (BE is)) (ADJP (RB too) (JJ low))))
fan speed is low
(S
  (NP (OBJECT (NN_OBJ (FAN fan))) (PROPERTY (NN_PROP (SPEED speed))))
  (VP (VBZ (BE is)) (ADJP (JJ low))))
temperature is high
(S
  (NP (PROPERTY (NN_PROP (TEMPERATURE temperature))))
  (VP (VBZ (BE is)) (ADJP (JJ high))))
humidity feels too high
(S
  (NP (PROPERTY (NN_PROP (HUMIDITY humidity))))
  (VP (VBZ (FEEL feels)) (ADJP (RB too) (JJ high))))
The room is very damp
(S
  (NP (OBJECT (DT the) (NN_OBJ (ENVIRONMENT room))))
  (VP (VBZ (BE is)) (ADJP (RB very) (JJ (HUMID damp)))))
It is burning
(S
  (NP (OBJECT (NN_OBJ (ENVIRONMENT it))))
  (VP (VBZ (BE is)) (ADJP (JJ (HOT burning)))))
it is so cold
(S
  (NP (OBJECT (NN_OBJ (ENVIRONMENT it))))
  (VP (VBZ (BE is)) (ADJP (RB so) (JJ (COLD cold)))))
the fan is not quite fast
(S
  (NP (OBJECT (DT the) (NN_OBJ (FAN fan))))
  (VP (VBZ

In [6]:
parse_tree = h.parse(text)

In [7]:
for t in parse_tree[0]:
    print(t)

(NP (OBJECT (USER i)))
(VP (VBZ (BE am)) (ADJP (JJ (COLD freezing))))


In [8]:
list(parse_tree[0])

[Tree('NP', [Tree('OBJECT', [Tree('USER', ['i'])])]),
 Tree('VP', [Tree('VBZ', [Tree('BE', ['am'])]), Tree('ADJP', [Tree('JJ', [Tree('COLD', ['freezing'])])])])]